### PS3 Solution CHEME 5440/7770 Walkthrough

In this problem we ask to you to compile a list of chemical reactions for a model of the [Urea cycle in human]((https://www.genome.jp/kegg-bin/show_pathway?hsa00220)).  
This solution is slightly different from the [PS3 solution posted last year](https://github.com/varnerlab/CHEME-7770-Cornell-S19) because in this problem we ask you to
update the flux bounds array, and include a dilution due to growth term, using the metabolite measurements and saturation constants from [Park et al](https://pubmed.ncbi.nlm.nih.gov/27159581/) or [BRENDA](https://www.brenda-enzymes.org). 

### How do I do the QA/QC balance check?
To check if the chemical reactions (contained in the ``Reactions.dat`` file) are balanced, issue the command:

  ```jl
    julia > include("CheckBalances.jl")
  ```
This will formulate the Atom matrix ``A``, and then compute the product of ``transpose(A)*S`` where ``S`` denotes the stoichiometric matrix (contained in the file ``Network.net``). Two arrays are produced: 1) ``epsilon_1`` describes the case when we do not consider the boundary species (we have to nothing/from nothing reactions). These reactions will appear unbalanced with all internal reactions balanced (first few cols); ii) however, ``epsilon_2`` describes the case when you do include the boundary species, all reactions are balanced (all cols aer zero).

In [5]:
# execute the solution -
include("CheckBalances.jl");

In [8]:
# Case 1: No external species: when do NOT include the external species, you are checking the balances inside the box 
# (the first 5 or 6 cols, depending upon how you implemented the reversible reactions) -
epsilon_1

6×21 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  4.0  …  21.0  0.0   0.0  -21.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  4.0  7.0     30.0  1.0   0.0  -29.0  -2.0  2.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0      7.0  0.0  -1.0   -7.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  5.0  4.0     17.0  0.0  -1.0  -17.0  -1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0      3.0  0.0   0.0   -3.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …   0.0  0.0   0.0    0.0   0.0  0.0

In [10]:
# Case 2: External species: including external species (metabolites that you are exchanging across the hypothetical boundary of the box) will give all zeros -
epsilon_2

6×21 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

### How do I estimate the fluxes?
To estimate the Urea flux, issue the command:

  ```jl
    julia > include("Solve.jl")
  ```
The ``Solve`` script formulates the constraints into a [Julia Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries-1) which is passed to the solver code contained in the ``Flux.jl`` file.
The solver returns a bunch of stuff; the ``objective_value`` and ``flux_array`` arguments contain the Urea flux and
the optimal flux distribution, respectively. The optimal flux that I calculated was approximately: 1.24 mmol/gDW-hr.

#### The solver returns a bunch of stuff, what is all that?

In [18]:
?calculate_optimal_flux_distribution

search: calculate_optimal_flux_distribution



```
calculate_optimal_flux_distribution(dictionary; min_flag=true)

Computes the optimal metabolic flux distribution given the constraints.

Inputs: a dictionary that contains:
`stoichiometric_matrix` - stoichiometric_matrix (M x R)
`default_flux_bounds_array` - (R x 2) array of the flux lower (L) and upper (U) bounds
`species_bounds_array` - (M x 2) array of species lower (L) and upper (U) bounds (if at steady-state, L = U = 0)
`objective_coefficient_array` - R x 1 vector holding indexes for objective vector

Outputs:
`objective_value` - value of the objective function at the optimum
`calculated_flux_array` - R x 1 flux array at the optimum
`dual_value_array` - R x 1 dual values
`uptake_array` - M x 1 array of S*v
`exit_flag` = 0 if optimal
`status_flag` = 5 if optimal
```


In [26]:
# execute the solution (w/H20 exchange) -
include("Solve.jl");

In [28]:
objective_value # (obj_value = c*bUrea)

-1.25496

### Theory: How do we incorporate metabolite information into the flux balance analysis constraints?
We can incorporate metabolite concentration measurements in two places, the material balances through the dilution due to growth term, and the flux bounds. 

#### Material balances

For the ith metabolite, at steady state, the material balance is given by:

$$
\sum_{j=1}^{R-1}\sigma_{ij}v_{j}+\left(\sigma_{i\mu}-x_{i}\right)\mu = 0\qquad{i=1,2,\dots,M}
$$

where $\sigma_{ij}$ denotes the stoichiometric coefficient for species $i$ and reaction $j$, $R$ denotes the number of reactions (where by convention the reaction at index $R$ is given by the growth reaction), $\sigma_{i\mu}$ denotes the stoichiometric coefficient for metabolite $i$ in the growth reaction, $x_{i}$ denotes the concentration measurement for species $i$, and $M$ denotes the number of species. In this case, $\sigma_{i\mu}=0$ for all metabiolites in the model (we assumed no metabolites were biomass precursors, this is clearly not correct for all of the metabolites in the model). Given this assumption the metabolite balances were given by:

$$
\sum_{j=1}^{R-1}\sigma_{ij}v_{j}-x_{i}\mu = 0\qquad{i=1,2,\dots,M}
$$

#### Flux bounds

In the flux balance analysis problem, each flux is bounded by a lower and upper bound. We can use metabolite information, and models for the rates of enzyme catalyzed reactionsto write better bounds. In particular, lets use multiple saturation kinetics to describe the jth rate which could depend upon many substrates:
$$
v_{j} = V^{max}_{j}\prod_{k\in{\mathcal{R}(j)}}\left(\frac{x_{k}}{K_{kj}+x_{k}}\right)
$$

where $\mathcal{R}(j)$ denotes the set of reactants that are involved in reaction j. Given this model (which is just one of many possible models), we can write the bound for each flux as:

$$
0\leq{v_{j}}\leq{V^{max}_{j}\prod_{k\in{\mathcal{R}(j)}}\left(\frac{x_{k}}{K_{kj}+x_{k}}\right)}
$$

for an irreversible flux, or:


$$
-{V^{max,-}_{j}\prod_{k\in{\mathcal{R^{-}}(j)}}\left(\frac{x_{k}}{K_{kj}+x_{k}}\right)}\leq{v_{j}}\leq{V^{max,+}_{j}\prod_{k\in{\mathcal{R^{+}}(j)}}\left(\frac{x_{k}}{K_{kj}+x_{k}}\right)}
$$

for the reversible case where the superscript(s) -,+ denote the reverse and forward directions, respectively